![image info](https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/images/banner_1.png)

# Proyecto 1 - Predicción de precios de vehículos usados

En este proyecto podrán poner en práctica sus conocimientos sobre modelos predictivos basados en árboles y ensambles, y sobre la disponibilización de modelos. Para su desasrrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 1: Predicción de precios de vehículos usados".

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 4. Sin embargo, es importante que avancen en la semana 3 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 4, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/b8be43cf89c540bfaf3831f2c8506614).

## Datos para la predicción de precios de vehículos usados

En este proyecto se usará el conjunto de datos de Car Listings de Kaggle, donde cada observación representa el precio de un automóvil teniendo en cuenta distintas variables como: año, marca, modelo, entre otras. El objetivo es predecir el precio del automóvil. Para más detalles puede visitar el siguiente enlace: [datos](https://www.kaggle.com/jpayne/852k-used-car-listings).

## Ejemplo predicción conjunto de test para envío a Kaggle

En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importación librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

%matplotlib inline
sns.set_style('darkgrid')
plt.style.use('ggplot')

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import cross_val_score 


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor

In [ ]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTrain_carListings.zip')
dataTesting = pd.read_csv('https://raw.githubusercontent.com/davidzarruk/MIAD_ML_NLP_2023/main/datasets/dataTest_carListings.zip', index_col=0)

In [ ]:
# Visualización datos de entrenamiento
print(dataTraining.info())
dataTraining.head()

In [ ]:
# Visualización datos de test
print(dataTesting.info())
dataTesting.head()

In [ ]:
def grafica_distribucion(df: pd.DataFrame):
    '''
    '''
    df_num = df.select_dtypes(exclude=[object]).copy()
    df_cat = df.select_dtypes(include=[object]).copy()

    var_num = df_num.columns.to_list()
    col = len(var_num)
    fig, axes = plt.subplots (1,col,figsize = (18,4))
    fig.suptitle('DISTRIBUCIÓN DE VARIABLES NUMERICAS')

    for i,j in enumerate (var_num):
        data = df_num[j]
        axes[i].hist(data,bins = 50,color = "green",alpha = 0.65, rwidth = 0.85);
        axes[i].set_xlabel(f'{j}')
        axes[1].set_ylabel('cantidad')
    
    sns.pairplot(df);
    

    var_cat= df_cat.columns.to_list()
    col = len(var_cat)
    fig, axes = plt.subplots (col,1,figsize = (17,7))
    fig.suptitle('DISTRIBUCIÓN DE VARIABLES CATAGÓRICAS')

    for i, j in enumerate(var_cat):
        data = df_cat[j].value_counts()
        sns.countplot(x=j, data=df_cat, ax=axes[i]);
        axes[i].set_xlabel(f'{j}')
        axes[i].set_ylabel('cantidad')
        axes[i].tick_params(axis='x', rotation=45,labelsize=7)

    return

def stats(df:pd.DataFrame):
    '''
    '''
    df_num = df.select_dtypes(exclude=[object]).copy()
    df_cat = df.select_dtypes(include=[object]).copy()

    for i,j in enumerate (df_cat.columns.tolist()):
        print(f'Variable: {j}')
        print(f'Cantidad de clases: {df_cat[j].nunique()}')
        print(f'Top 10 de las clases: {Counter(df_cat[j]).most_common(10)}')
        print('------------------------------------')
        
    return df_num.describe().T 

def remover(df, columns, threshold=3):
    '''
    '''
    df_no_outliers = pd.DataFrame()
    
    for column in columns:
        z_scores = np.abs((df[column] - df[column].mean()) / df[column].std())
        df_filtered = df[z_scores <= threshold]
        df_no_outliers = pd.concat([df_no_outliers, df_filtered], ignore_index=True)
    
    return df_no_outliers

def metrics(test,predd):
      RMSE = np.sqrt(mean_squared_error(test,predd))
      return print(f"La métrica de evaluación del modelo regresión es:\nRMSE : {RMSE:,.6f}")

def metrics_CV(model,X,y):
      mse_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
      mean_mse = -np.mean(mse_scores)
      RMSE = np.sqrt(mean_mse)
      return RMSE

In [ ]:
stats(dataTraining)

In [ ]:
dataTraining['Make'].value_counts()

In [ ]:
grafica_distribucion(dataTraining)

In [ ]:
dataTraining_org = dataTraining.copy()
dataTraining_org['Make-Model'] = dataTraining_org['Make'] + '-' + dataTraining_org['Model']
dataTraining_org.drop(columns=['Make', 'Model'], inplace=True)

In [ ]:
dataTraining_org = remover(dataTraining_org, ['Mileage'], threshold=100)
#dataTraining_org['Price'] = np.log(dataTraining_org['Price'])  
sns.pairplot(dataTraining_org);

In [ ]:
X = dataTraining_org.drop(['Price'], axis=1)
y = dataTraining_org['Price']

data_num = X.select_dtypes(exclude=[object]).copy()
data_cat = X.select_dtypes(include=[object]).copy()

scaler = StandardScaler()
data_num_scaled = scaler.fit_transform(data_num)

#decomposer = PLSRegression(n_components=2).fit(data_num_scaled,y)
#data_num_pls = decomposer.transform(data_num_scaled)

decomposer = PCA(n_components=2).fit(data_num_scaled)
data_num_pca = decomposer.transform(data_num_scaled)

for i in data_cat.columns:
    data_cat[i] = data_cat[i].astype('category')

data_cat_encoded = pd.get_dummies(data_cat,columns=data_cat.columns, drop_first=True)
#encoder = OneHotEncoder(handle_unknown='ignore', drop='first').fit(data_cat)
#data_cat_encoded = encoder.transform(data_cat).toarray()


In [ ]:

X_training = np.concatenate((data_num_pca, data_cat_encoded), axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_training, y, test_size=0.20, random_state=42)

X_train = pd.DataFrame(X_train, columns=['PCA1', 'PCA2'] + data_cat_encoded.columns.tolist())
X_test = pd.DataFrame(X_test, columns=['PCA1', 'PCA2'] + data_cat_encoded.columns.tolist())

X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
range_depth = [int(x) for x in np.linspace(1,10,10)]
RMSE_scores = []

params = {'learning_rate': 0.5,
          'n_estimators': 500}   

for depth in range_depth:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, max_depth=depth, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

depth = range_depth[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {depth}')

In [ ]:
range_leaves = [int(x) for x in np.linspace(1, 50, 50)]
RMSE_scores = []

params = {'learning_rate': 0.5,
          'n_estimators': 500,
          'max_depth': depth}   

for leaves in range_leaves:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, max_leaves=leaves, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

leaves = range_leaves[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {leaves}')

In [ ]:
#range_gamma= [float(x) for x in np.linspace(0, 1.0, 50)]
range_gamma= [0.001,0.1]
RMSE_scores = []

params = {'learning_rate': 15,
          'n_estimators': 500,
          'max_depth': depth,
          'max_leaves': leaves} 

for gamma_ in range_gamma:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, gamma=gamma_, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

gamma_ = range_gamma[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {gamma_}')

In [ ]:
X_test

In [ ]:
range_subsample = [float(x) for x in np.linspace(0, 1.0, 50)]
RMSE_scores = []

params = {'learning_rate': 15,
          'n_estimators': 500,
          'max_depth': depth,
          'max_leaves': leaves,
          'gamma': gamma_}   

for subsample in range_subsample:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, subsample=subsample, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

subsample = range_subsample[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {subsample}')

In [ ]:
range_colsample = [float(x) for x in np.linspace(0, 1.0, 50)]
RMSE_scores = []

params = {'learning_rate': 15,
          'n_estimators': 500,
          'max_depth': depth,
          'gamma': gamma_,
          'max_leaves': leaves,
          'subsample': subsample}   

for colsample in range_colsample:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, colsample_bytree=colsample, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

colsample = range_colsample[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {colsample}')

In [ ]:
range_regularization = [float(x) for x in np.linspace(0, 5, 100)]
RMSE_scores = []

params = {'learning_rate': 15,
          'n_estimators': 500,
          'max_depth': depth,
          'gamma': gamma_,
          'max_leaves': leaves,
          'subsample': subsample,
          'colsample_bytree': colsample}  

for regul in range_regularization:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, reg_alpha=regul, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

regul = range_regularization[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {regul}')

In [ ]:
range_learning = [float(x) for x in np.linspace(0.01, 0.1, 100)]
RMSE_scores = []

params = {'n_estimators': 500,
          'max_depth': depth,
          'gamma': gamma_,
          'max_leaves': leaves,
          'subsample': subsample,
          'colsample_bytree': colsample,
          'reg_alpha': regul}  

for learning in range_learning:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, learning_rate=learning)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

learning = range_learning[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {learning}')

In [ ]:
range_estimators = [int(x) for x in np.linspace(150, 350, 100)]
RMSE_scores = []

params = {'max_depth': depth,
          'gamma': gamma_,
          'max_leaves': leaves,
          'subsample': subsample,
          'colsample_bytree': colsample,
          'reg_alpha': regul,
          'learning_rate': learning}  

for estimator in range_estimators:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, n_estimators=estimator, learning_rate=learning)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

estimators = range_estimators[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {estimators}')

In [ ]:
range_gamma= [float(x) for x in np.linspace(0, 1.0, 50)]
RMSE_scores = []

params = {'max_depth': depth,
          'max_leaves': leaves,
          'subsample': subsample,
          'colsample_bytree': colsample,
          'reg_alpha': regul,
          'learning_rate': learning,
          'n_estimators': estimators}  

for gamma_ in range_gamma:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, gamma=gamma_, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

gamma_ = range_gamma[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {gamma_}')

In [ ]:
range_leaves = [int(x) for x in np.linspace(1, 50, 50)]
RMSE_scores = []

params = {'max_depth': depth,
          'subsample': subsample,
          'colsample_bytree': colsample,
          'reg_alpha': regul,
          'learning_rate': learning,
          'n_estimators': estimators,
          'gamma': gamma_}   

for leaves in range_leaves:
    model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, max_leaves=leaves, **params)
    model_xgb.fit(X_train, y_train)
    prediction = model_xgb.predict(X_test)
    RMSE_scores.append(np.sqrt(mean_squared_error(y_test, prediction)))

leaves = range_leaves[np.argmin(RMSE_scores)]
print(f'El valor óptimo es: {leaves}')

In [ ]:
params = {'learning_rate': 0.9252525252525252,
           'n_estimators': 198,
           'max_depth': 6,
           'max_leaves': 37, 
           'subsample': 1.0, 
           'colsample_bytree': 1.0,
           'gamma': 0.0, 
           'reg_alpha': 2.2222222222222223}

model_xgb = XGBRegressor(objective ='reg:squarederror', eval_metric='rmse',random_state=42, n_jobs=-1, ** params)

model_xgb.fit(X_train, y_train)
prediction = model_xgb.predict(X_test)

metrics(y_test, prediction)

#### with the complete set of data (threshold = 100)



#### with the complete set of data (threshold = 30)







In [ ]:
model_xgb.fit(X_training, y)

data_num_ = dataTesting.select_dtypes(exclude=[object]).copy()
data_cat_ = dataTesting.select_dtypes(include=[object]).copy()



data_num_scaled_ = scaler.transform(data_num_)

data_num_pca_ = decomposer.transform(data_num_scaled_)

for i in data_cat_.columns:
    data_cat_[i] = data_cat_[i].astype('category')

data_cat_encoded_ = pd.get_dummies(data_cat_,columns=data_cat_.columns, drop_first=True)
#data_cat_encoded_ = encoder.transform(data_cat_).toarray()

X_testing = np.concatenate((data_num_pca_, data_cat_encoded_), axis=1)
X_testing = pd.DataFrame(X_testing, columns=['PCA1', 'PCA2'] + data_cat_encoded_.columns.tolist()) 
X_testing['Make_Freightliner'] = 0
X_testing = X_testing[X_train.columns.tolist()]

y_prediction = model_xgb.predict(X_testing)

In [ ]:
y_predd = pd.DataFrame(y_prediction, columns=['Price'])
y_predd.to_csv('20240422_woo_pca_comp_.csv', index_label='ID')

In [ ]:
# Predicción del conjunto de test - acá se genera un número aleatorio como ejemplo
np.random.seed(42)
y_pred = pd.DataFrame(np.random.rand(dataTesting.shape[0]) * 75000 + 5000, index=dataTesting.index, columns=['Price'])

In [ ]:
# Guardar predicciones en formato exigido en la competencia de kaggle
y_pred.to_csv('test_submission.csv', index_label='ID')
y_pred.head()